In [1]:
import numpy as np
from pychunkedgraph.graph import ChunkedGraph, ChunkedGraphMeta
from pychunkedgraph.graph.meta import ChunkedGraphMeta, GraphConfig, DataSource

ds = DataSource(
  EDGES= "gs://chunkedgraph/minnie65_0/edges",
  COMPONENTS= "gs://chunkedgraph/minnie65_0/components",
  WATERSHED= "gs://microns-seunglab/minnie65/ws_minnie65_0"
)
gc = GraphConfig(
  CHUNK_SIZE= [256, 256, 512]
)
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

In [2]:
edges = cg.read_chunk_edges([cg.get_chunk_id(layer=2,x=50,y=100,z=0)])
edges = edges["between"].get_pairs()
nodes = np.unique(edges)
parents = cg.get_roots(nodes)
d = dict(zip(parents, nodes))
print(d)

{864691138338302132: 75582971913961693, 864691137629078617: 75512534450307707, 864691137995403800: 75582903194484819, 864691137995404568: 75582834475009044, 864691138741942725: 75582971913961542, 864691137995404824: 75582903194484963, 864691136334203586: 75582971913961503}


In [3]:
edges = np.array([[75582971913961693, 75582971913961503]])
print(cg.get_cross_chunk_edges_layer(edges))
print(cg.get_parents(edges[0]))

[1]
[147640565951889409 147640565951889410]


In [4]:
parents = np.uint64(edges[0][0])
while parents and parents.size:
    parents = cg.get_parent(parents)
    print(parents)

147640565951889409
219704963218014230
291635563662936192
363718446468361184
435828817064694903
506866064312035921
576742227280767262
864691138338302132
None


In [5]:
from pychunkedgraph.graph.edits import add_edges
from pychunkedgraph.graph.utils.context_managers import TimeIt

edges = np.array(edges, dtype=np.uint64)
with TimeIt("add_edge"):
    print(add_edges(cg, atomic_edges=edges))

Cannot find root id 363718446468300827, 6, 2020-01-31 15:23:37.857000+00:00
Cannot find root id 576742227280134171, 12, 2020-01-31 15:23:43.013000+00:00
[864691128455135276]
add_edge 8.697100400924683


In [6]:
cg.node_hierarchy

{147640565951889452: 147640565951889452:219704963218014270:[75582971913961473 75582971913961891 75582971913961912 75582971913961915
  75582971913961925 75582971913961926 75582971913961934 75582971913961954
  75582971913961956 75582971913961966 75582971913961972 75582971913961983
  75582971913962060 75582971913962076 75582971913962104 75582971913962136
  75582971913961884 75582971913961850 75582971913961923 75582971913961830
  75582971913961509 75582971913961488 75582971913961513 75582971913961693
  75582971913961501 75582971913961503],
 75582971913961473: 75582971913961473:147640565951889452:[],
 75582971913961891: 75582971913961891:147640565951889452:[],
 75582971913961912: 75582971913961912:147640565951889452:[],
 75582971913961915: 75582971913961915:147640565951889452:[],
 75582971913961925: 75582971913961925:147640565951889452:[],
 75582971913961926: 75582971913961926:147640565951889452:[],
 75582971913961934: 75582971913961934:147640565951889452:[],
 75582971913961954: 75582971913

In [12]:
cg.meta.cv.info

{'data_type': 'uint64',
 'num_channels': 1,
 'scales': [{'chunk_sizes': [[128, 128, 16]],
   'encoding': 'raw',
   'key': '8_8_40',
   'resolution': [8, 8, 40],
   'size': [182150, 125976, 12998],
   'voxel_offset': [31744, 43778, 14884]}],
 'type': 'segmentation'}